# 실습 10. 

**from dgl.nn import SAGEConv** 를 직접 구현하고, 이를 이용하여 GraphSAGE 모델을 학습시켜보기

In [1]:
!pip install dgl

     |████████████████████████████████| 4.2MB 5.8MB/s 


In [2]:
import numpy as np                        
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.data import CoraGraphDataset
from sklearn.metrics import f1_score
import dgl.function as fn

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [68]:
# 하이퍼파라미터 정의
learningRate = 1e-2
numEpochs = 50
numHiddenDim = 128
numLayers = 2
weightDecay = 5e-4

In [69]:
'''
    Cora 데이터셋은 2708개의 논문(노드), 10556개의 인용관계(엣지)로 이루어졌습니다. 
    NumFeat은 각 노드를 나타내는 특성을 말합니다. 
    Cora 데이터셋은 각 노드가 1433개의 특성을 가지고, 개개의 특성은 '1'혹은 '0'으로 나타내어지며 특정 단어의 논문 등장 여부를 나타냅니다.
    즉, 2708개의 논문에서 특정 단어 1433개를 뽑아서, 1433개의 단어의 등장 여부를 통해 각 노드를 표현합니다.
    
    노드의 라벨은 총 7개가 존재하고, 각 라벨은 논문의 주제를 나타냅니다
    [Case_Based, Genetic_Algorithms, Neural_Networks, Probabilistic_Methods, Reinforcement_Learning, Rule_Learning, Theory]

    2708개의 노드 중, 학습에는 140개의 노드를 사용하고 모델을 테스트하는 데에는 1000개를 사용합니다.
    본 실습에서는 Validation을 진행하지않습니다.

    요약하자면, 앞서 학습시킬 모델은 Cora 데이터셋의 
    [논문 내 등장 단어들, 논문들 사이의 인용관계]를 활용하여 논문의 주제를 예측하는 모델입니다.
'''

# Cora Graph Dataset 불러오기
G = CoraGraphDataset()
numClasses = G.num_classes

G = G[0]
# 노드들의 feauture & feature의 차원
features = G.ndata['feat']
inputFeatureDim = features.shape[1]

# 각 노드들의 실제 라벨
labels = G.ndata['label']

# 학습/테스트에 사용할 노드들에 대한 표시
trainMask = G.ndata['train_mask']        
testMask = G.ndata['test_mask']

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [129]:
trainMask.size()

torch.Size([2708])

In [70]:
# 모델 학습 결과를 평가할 함수
def evaluateTrain(model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

def evaluateTest(model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        macro_f1 = f1_score(labels, indices, average = 'macro')
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels), macro_f1

In [71]:
def train(model, lossFunction, features, labels, trainMask, optimizer, numEpochs):
    executionTime=[]
    
    for epoch in range(numEpochs):
        model.train()

        startTime = time.time()
            
        logits = model(features)                                    # 포워딩
        loss = lossFunction(logits[trainMask], labels[trainMask])   # 모델의 예측값과 실제 라벨을 비교하여 loss 값 계산

        optimizer.zero_grad()                                       
        loss.backward()
        optimizer.step()

        executionTime.append(time.time() - startTime)

        acc = evaluateTrain(model, features, labels, trainMask)

        print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f}".format(epoch, np.mean(executionTime), loss.item(), acc))

def test(model, feautures, labels, testMask):
    acc, macro_f1 = evaluateTest(model, features, labels, testMask)
    print("Test Accuracy {:.4f}".format(acc))
    print("Test macro-f1 {:.4f}".format(macro_f1))

<image src = https://user-images.githubusercontent.com/48677363/109593959-4c736680-7b55-11eb-8982-89367a4ae135.png width = 700>


In [72]:
class SAGEConv(nn.Module):
    """
    in_feats: 인풋 feature의 사이즈
    out_feats: 아웃풋 feature의 사이즈
    activation: None이 아니라면, 노드 피쳐의 업데이트를 위해서 해당 activation function을 적용한다.
    """
    '''
        ref:
        https://arxiv.org/pdf/1706.02216.pdf 
        https://docs.dgl.ai/en/0.4.x/_modules/dgl/nn/pytorch/conv/sageconv.html
    '''
    
    def __init__(self, in_feats, out_feats, activation):
        super(SAGEConv, self).__init__()
        self._in_feats = in_feats # 입력 차원
        self._out_feats = out_feats # 출력 차원
        self.activation = activation # 활성화 함수

        self.W = nn.Linear(in_feats+in_feats, out_feats, bias=True) # 집계 과정 신경망, concat을 통해 차원이 2배가 되므로 in_feats + in_feats

    def forward(self, graph, feature):
        graph.ndata['h'] = feature # 그래프 데이터
        # dgl.function.sum: https://docs.dgl.ai/en/0.4.x/generated/dgl.function.sum.html?                                                
        graph.update_all(fn.copy_src('h', 'm'), fn.sum('m', 'neigh')) # h -> m 복사, sum -> neigh로 저장, fn.sum -> 이웃들 집계 함수로 예상됨
        # Aggregate & Noramlization
        degs = graph.in_degrees().to(feature) # 2708 정점에 대한 차수 계산
        hkNeigh = graph.ndata['neigh'] / degs.unsqueeze(-1) # 2708 정점에 대한 정보 / 차수
        hk = self.W(torch.cat((graph.ndata['h'], hkNeigh), dim = -1))
                
        if self.activation != None:
            hk = self.activation(hk)

        return hk

In [88]:
G.ndata['h'].size()

torch.Size([2708, 128])

In [118]:
G.edges()[0][:2]

tensor([0, 0])

In [119]:
G.edges()[1][:2]

tensor([ 633, 1862])

In [126]:
sum(G.ndata['h'][[633, 1862]])

tensor([ 0.0000,  0.0000,  7.5935,  5.6773,  0.2287,  7.4217,  0.0000,  3.1079,
         9.5192,  0.0000,  7.6594,  7.6557,  0.2052,  5.2661,  3.7209,  0.7848,
         6.3562,  0.5177,  3.9495,  0.0000,  7.1148,  0.0551,  0.0000,  8.3314,
         0.0000,  0.3364,  9.5248,  0.0000,  4.0757,  7.1054,  0.0000,  3.4573,
         2.7561,  0.0000,  3.5737,  2.0959,  5.7971,  4.2305,  0.7770,  0.0000,
         0.9451,  0.0000,  9.9228,  9.2247,  0.0000,  0.0000,  0.1442,  4.2094,
         7.4494,  0.0000,  0.1440,  0.0000,  2.9801,  0.3242,  0.0000,  0.0000,
         0.2546,  0.0000,  6.1822,  3.4752,  4.3285,  0.0000,  7.7517,  2.7793,
         6.7248,  7.3374,  6.6461,  4.0571,  5.7680,  1.3371,  0.0000,  0.0000,
         6.1847,  0.1285,  0.0000,  5.3256,  3.5583,  0.2360,  0.0000,  0.0000,
         2.2174,  0.0000,  0.1632,  0.0000,  0.1884,  0.0000,  0.8362,  1.8996,
         0.0000,  3.6003,  2.1996,  0.0000,  0.0000,  5.1630,  3.8910,  0.4275,
         0.8291,  5.2436,  0.0000,  0.00

In [91]:
G.update_all(fn.copy_src('h', 'm'), fn.sum('m', 'temp'))

In [127]:
G.ndata['temp'][0]

tensor([ 0.0000,  0.0000, 12.0398,  7.9994,  0.2287, 11.5753,  0.0000,  5.0354,
        14.5488,  0.0000, 11.2258, 11.0150,  0.9598,  8.0838,  6.1319,  0.7901,
         9.0166,  0.7851,  5.8554,  0.0000, 10.1574,  0.0621,  0.0000, 12.2071,
         0.0000,  0.5008, 15.1316,  0.0000,  6.2323, 10.1334,  0.0000,  5.6201,
         5.1340,  0.0000,  5.5848,  2.8209,  8.4250,  6.0940,  1.2399,  0.0000,
         1.4477,  0.0000, 15.3222, 14.7036,  0.0000,  0.0000,  0.2061,  6.0429,
        10.7541,  0.0000,  0.1440,  0.0000,  4.7869,  0.4843,  0.0000,  0.0000,
         0.4219,  0.0000,  8.8751,  5.6482,  6.9866,  0.0000, 12.3366,  4.6922,
        10.3718, 11.4757,  9.4619,  6.6935,  8.2274,  1.8768,  0.0000,  0.0000,
         9.4413,  0.2001,  0.0000,  8.0104,  5.1267,  0.2360,  0.0000,  0.0000,
         3.8186,  0.0000,  0.1632,  0.0257,  0.5033,  0.0000,  1.2739,  2.6961,
         0.0000,  6.1235,  3.2746,  0.0000,  0.0000,  8.3087,  5.6106,  0.6784,
         1.1315,  7.3462,  0.0000,  0.00

In [81]:
G.in_degrees().to(features).size(), G.in_degrees().to(features)

(torch.Size([2708]), tensor([3., 3., 5.,  ..., 1., 4., 4.]))

In [85]:
G.ndata['neigh'].size()

torch.Size([2708, 128])

In [73]:

class GraphSAGE(nn.Module):
    '''
        graph               : 학습할 그래프
        inFeatDim           : 데이터의 feature의 차원
        numHiddenDim        : 모델의 hidden 차원
        numClasses          : 예측할 라벨의 경우의 수
        numLayers           : 인풋, 아웃풋 레이어를 제외하고 중간 레이어의 갯수
        activationFunction  : 활성화 함수의 종류
    '''
    def __init__(self, graph, inFeatDim, numHiddenDim, numClasses, numLayers, activationFunction):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.graph = graph

        # 인풋 레이어
        self.layers.append(SAGEConv(inFeatDim, numHiddenDim, activationFunction))
       
        # 히든 레이어
        for i in range(numLayers):
            self.layers.append(SAGEConv(numHiddenDim, numHiddenDim, activationFunction))
        
        # 출력 레이어
        self.layers.append(SAGEConv(numHiddenDim, numClasses, activation=None))

    def forward(self, features):
        x = features
        for layer in self.layers:
            x = layer(self.graph, x)
        return x

In [74]:
# 모델 생성
model = GraphSAGE(G, inputFeatureDim, numHiddenDim, numClasses, numLayers, F.relu)
print(model)

lossFunction = torch.nn.CrossEntropyLoss()

# 옵티마이저 초기화
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecay)

GraphSAGE(
  (layers): ModuleList(
    (0): SAGEConv(
      (W): Linear(in_features=2866, out_features=128, bias=True)
    )
    (1): SAGEConv(
      (W): Linear(in_features=256, out_features=128, bias=True)
    )
    (2): SAGEConv(
      (W): Linear(in_features=256, out_features=128, bias=True)
    )
    (3): SAGEConv(
      (W): Linear(in_features=256, out_features=7, bias=True)
    )
  )
)


In [75]:
train(model, lossFunction, features, labels, trainMask, optimizer, numEpochs)

Epoch 00000 | Time(s) 0.1759 | Loss 1.9470 | Accuracy 0.1429
Epoch 00001 | Time(s) 0.1761 | Loss 1.9461 | Accuracy 0.1500
Epoch 00002 | Time(s) 0.1752 | Loss 1.9431 | Accuracy 0.2929
Epoch 00003 | Time(s) 0.1741 | Loss 1.9377 | Accuracy 0.3429
Epoch 00004 | Time(s) 0.1727 | Loss 1.9168 | Accuracy 0.3571
Epoch 00005 | Time(s) 0.1748 | Loss 1.8565 | Accuracy 0.4643
Epoch 00006 | Time(s) 0.1740 | Loss 1.7231 | Accuracy 0.4643
Epoch 00007 | Time(s) 0.1740 | Loss 1.4973 | Accuracy 0.4214
Epoch 00008 | Time(s) 0.1735 | Loss 1.2646 | Accuracy 0.4000
Epoch 00009 | Time(s) 0.1732 | Loss 1.2861 | Accuracy 0.4286
Epoch 00010 | Time(s) 0.1728 | Loss 1.0539 | Accuracy 0.4000
Epoch 00011 | Time(s) 0.1727 | Loss 1.1192 | Accuracy 0.7571
Epoch 00012 | Time(s) 0.1727 | Loss 0.7383 | Accuracy 0.6429
Epoch 00013 | Time(s) 0.1725 | Loss 0.7361 | Accuracy 0.7071
Epoch 00014 | Time(s) 0.1722 | Loss 0.7122 | Accuracy 0.8714
Epoch 00015 | Time(s) 0.1719 | Loss 0.5181 | Accuracy 0.9429
Epoch 00016 | Time(s) 0.

In [76]:
test(model, features, labels, testMask)

Test Accuracy 0.7500
Test macro-f1 0.7507
